In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, f1_score
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from sklearn.model_selection import train_test_split


pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


In [20]:
def compute_performance_metrics(y_test, y_pred, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1_score_value = f1_score(y_test, y_pred)
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"{model_name} Recall: {recall}")
    print(f"{model_name} F1 Score: {f1_score_value}")

def compute_classification_metric(dataset,predictions, label_name_v, favorable_label_v, unfavorable_label_v, privileged_attribute, unprivileged_attribute):
    features = [privileged_attribute,unprivileged_attribute] # We want to check the fairness level regarding the protected attribute "sex"

    # This is the object made of the original dataset
    aif_sex_dataset = BinaryLabelDataset( # Base class for all structured datasets with binary labels.
            df=dataset,
            favorable_label=favorable_label_v, # This means that a prediction is biased toward the privileged attribute if its value is 1 (True)
            unfavorable_label=unfavorable_label_v,
            label_names=[label_name_v],
            protected_attribute_names=features,
            privileged_protected_attributes=[privileged_attribute], # here we tell AIF that we want to check for predictions
        )

    # We do the same thing but with the predictions dataset
    aif_sex_pred = BinaryLabelDataset(
            df=predictions,
            favorable_label=favorable_label_v,
            unfavorable_label=unfavorable_label_v,
            label_names=[label_name_v],
            protected_attribute_names=features,
            privileged_protected_attributes=[privileged_attribute],
        )

    sex_privileged_group = [{privileged_attribute: 1, unprivileged_attribute: 0}] # The privileged group is made of males (sex_Male = True)
    sex_unprivileged_group = [{unprivileged_attribute: 1, privileged_attribute: 0}] # The unprivileged group is made of females (sex_Female = True)

    # We provide the ClassificationMetric object with all the information needed:
    # aif_sex_dataset - The original test set
    # aif_sex_pred - A dataset containing the predictions of the model
    # sex_privileged_group - The privileged group
    # sex_unprivileged_group - The unprivileged group
    fairness_metrics = ClassificationMetric(dataset=aif_sex_dataset,
                            classified_dataset=aif_sex_pred,
                            unprivileged_groups=sex_unprivileged_group,
                            privileged_groups=sex_privileged_group)
    
    return fairness_metrics

def compute_fairness_metrics(fairness_metrics: ClassificationMetric):
    # Values less than 0 indicate that privileged group has higher
    # proportion of predicted positive outcomes than unprivileged group.
    # Value higher than 0 indicates that unprivileged group has higher proportion
    # of predicted positive outcomes than privileged group.
    SPD = round(fairness_metrics.statistical_parity_difference(),3)

    # Measures the deviation from the equality of opportunity, which means that the same
    # proportion of each population receives the favorable outcome. This measure must be equal to 0 to be fair.
    EOD = round(fairness_metrics.equal_opportunity_difference(),3)

    # Average of difference in False Positive Rate and True Positive Rate for unprivileged and privileged groups
    # A value of 0 indicates equality of odds, which means that samples in both the privileged and unprivileged
    # groups have the same probability of being classified positively.
    AOD = round(fairness_metrics.average_odds_difference(),3)

    print(f"Statistical Parity Difference (SPD): {SPD}")
    print(f"Equal Opportunity Difference (EOD): {EOD}")
    print(f"Average Odds Difference: {AOD}")

In [3]:
dataset_path = 'datasets/compas-scores.csv'
df_raw = pd.read_csv(dataset_path) 

In [4]:
df_raw.shape

(11757, 47)

In [5]:
df_raw.columns

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'num_r_cases',
       'r_case_number', 'r_charge_degree', 'r_days_from_arrest',
       'r_offense_date', 'r_charge_desc', 'r_jail_in', 'r_jail_out',
       'is_violent_recid', 'num_vr_cases', 'vr_case_number',
       'vr_charge_degree', 'vr_offense_date', 'vr_charge_desc',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'type_of_assessment', 'decile_score.1',
       'score_text', 'screening_date'],
      dtype='object')

In [6]:
df_raw.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,vr_offense_date,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,type_of_assessment,decile_score.1,score_text,screening_date
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,NaN,NaN,Risk of Violence,1,Low,2013-08-14,Risk of Recidivism,1,Low,2013-08-14
1,2,michael ryan,michael,ryan,2014-12-31,Male,1985-02-06,31,25 - 45,Caucasian,...,NaN,NaN,Risk of Violence,2,Low,2014-12-31,Risk of Recidivism,5,Medium,2014-12-31
2,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,2013-07-05,Felony Battery (Dom Strang),Risk of Violence,1,Low,2013-01-27,Risk of Recidivism,3,Low,2013-01-27
3,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,NaN,NaN,Risk of Violence,3,Low,2013-04-14,Risk of Recidivism,4,Low,2013-04-14
4,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,NaN,NaN,Risk of Violence,6,Medium,2013-01-13,Risk of Recidivism,8,High,2013-01-13


In [7]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11757 entries, 0 to 11756
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       11757 non-null  int64  
 1   name                     11757 non-null  object 
 2   first                    11757 non-null  object 
 3   last                     11757 non-null  object 
 4   compas_screening_date    11757 non-null  object 
 5   sex                      11757 non-null  object 
 6   dob                      11757 non-null  object 
 7   age                      11757 non-null  int64  
 8   age_cat                  11757 non-null  object 
 9   race                     11757 non-null  object 
 10  juv_fel_count            11757 non-null  int64  
 11  decile_score             11757 non-null  int64  
 12  juv_misd_count           11757 non-null  int64  
 13  juv_other_count          11757 non-null  int64  
 14  priors_count          

In [8]:
columns_to_drop = ['num_r_cases',
'r_case_number',
'r_charge_degree',
'r_days_from_arrest',
'r_offense_date',
'r_charge_desc',
'r_jail_in',
'r_jail_out',
'is_violent_recid',
'num_vr_cases',
'vr_case_number',
'vr_charge_degree',
'vr_offense_date',
'vr_charge_desc',
'v_type_of_assessment',
'v_decile_score',
'v_score_text',
'v_screening_date',
'c_arrest_date',
'decile_score.1',
'screening_date',
'id',
'compas_screening_date',
'type_of_assessment',
'dob',
'name',
'first',
'last',
'score_text',
'juv_fel_count',
'juv_misd_count',
'juv_other_count',
'c_jail_in',
'c_jail_out',
'c_offense_date',
'c_case_number']

df_raw = df_raw.drop(columns=columns_to_drop)
df_raw = df_raw.dropna()


In [9]:
categorical_columns = ["sex", "age_cat", "race", "c_charge_degree", "c_charge_desc"]
df_raw = pd.get_dummies(df_raw, columns=categorical_columns)

In [10]:
X = df_raw.drop(columns="is_recid")
y = df_raw["is_recid"]

# Define four sets and apply the function
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, # 0.2 indicates a test set size of 20%
                                                    random_state=42)

In [11]:
dt_clf = DecisionTreeClassifier(random_state=42)

# The fit function will do the trick
dt_clf.fit(X_train, y_train)

# After the training phase, the model will be tested by predicting the values on the test set
dt_predictions = dt_clf.predict(X_test)

compute_performance_metrics(y_test,dt_predictions,"Decision Tree")

Decision Tree Accuracy: 0.6305581835383159
Decision Tree Recall: 0.39662447257383965
Decision Tree F1 Score: 0.41933085501858736


In [12]:
svm_classifier = make_pipeline(StandardScaler(), SVC(kernel='linear'))

# Addestra il classificatore sui dati di training
svm_classifier.fit(X_train, y_train)

# Fai delle predizioni sui dati di test
svm_pred = svm_classifier.predict(X_test)

compute_performance_metrics(y_test,svm_pred,"SVM")

SVM Accuracy: 0.684484389782403
SVM Recall: 0.2222222222222222
SVM F1 Score: 0.3214649033570702


In [13]:
rf_classifier = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth = None, random_state=42)

rf_classifier.fit(X_train, y_train)

rf_predictions = rf_classifier.predict(X_test)

compute_performance_metrics(y_test,rf_predictions,"Random Forest")

Random Forest Accuracy: 0.6764427625354777
Random Forest Recall: 0.36146272855133615
Random Forest F1 Score: 0.4290484140233723


In [14]:
pd.set_option('display.max_columns', None)
df_raw.head()

,age,decile_score,priors_count,days_b_screening_arrest,c_days_from_compas,is_recid,sex_Female,sex_Male,age_cat_25 - 45,age_cat_Greater than 45,age_cat_Less than 25,race_African-American,race_Asian,race_Caucasian,race_Hispanic,race_Native American,race_Other,c_charge_degree_F,c_charge_degree_M,c_charge_degree_O,c_charge_desc_Abuse Without Great Harm,c_charge_desc_Agg Abuse Elderlly/Disabled Adult,c_charge_desc_Agg Assault W/int Com Fel Dome,c_charge_desc_Agg Battery Cause Bodily,c_charge_desc_Agg Battery Grt/Bod/Harm,c_charge_desc_Agg Fleeing And Eluding,c_charge_desc_Agg Fleeing and Eluding,c_charge_desc_Agg Fleeing/Eluding High Speed,"c_charge_desc_Aggr Child Abuse-Torture,Punish",c_charge_desc_Aggrav Battery w/Deadly Weapon,c_charge_desc_Aggrav Child Abuse-Agg Battery,c_charge_desc_Aggrav Child Abuse-Causes Harm,c_charge_desc_Aggrav Stalking After Injunctn,c_charge_desc_Aggravated Animal Cruelty,c_charge_desc_Aggravated Assault,c_charge_desc_Aggravated Assault W/Dead Weap,c_charge_desc_Aggravated Assault W/dead Weap,c_charge_desc_Aggravated Assault W/o Firearm,c_charge_desc_Aggravated Assault w/Firearm,c_charge_desc_Aggravated Battery,c_charge_desc_Aggravated Battery (Firearm),c_charge_desc_Aggravated Battery (Firearm/Actual Possession),c_charge_desc_Aggravated Battery / Pregnant,c_charge_desc_Aggravated Battery On 65/Older,c_charge_desc_Aggravated Manslaughter Child,c_charge_desc_Aggress/Panhandle/Beg/Solict,c_charge_desc_Aide/Abet Prostitution Lewdness,c_charge_desc_Aiding Escape,c_charge_desc_Alcohol Bev Under Age Of 21,c_charge_desc_Alcoholic Beverage Violation-FL,c_charge_desc_Armed Kidnapping,c_charge_desc_Armed Trafficking in Cannabis,c_charge_desc_Arson II (Vehicle),c_charge_desc_Arson in the First Degree,c_charge_desc_Assault,c_charge_desc_Assault Law Enforcement Officer,c_charge_desc_Att Burgl Conv Occp,c_charge_desc_Att Burgl Occp Dwelling,c_charge_desc_Att Burgl Struc/Conv Dwel/Occp,c_charge_desc_Att Burgl Unoccupied Dwel,c_charge_desc_Att Robbery Sudd Snatch No Weap,c_charge_desc_Att Tamper w/Physical Evidence,c_charge_desc_Attempt Armed Burglary Dwell,c_charge_desc_Attempt Burglary (Struct),c_charge_desc_Attempt Murder in the First Degree,c_charge_desc_Attempt Sexual Batt / Vict 12+,c_charge_desc_Attempt/Solicitation/Conspiracy,c_charge_desc_Attempted Aggravated Battery,c_charge_desc_Attempted Burg/Convey/Unocc,c_charge_desc_Attempted Burg/struct/unocc,c_charge_desc_Attempted Deliv Control Subst,c_charge_desc_Attempted Robbery No Weapon,c_charge_desc_Attempted Robbery Weapon,c_charge_desc_Attempted Robbery Firearm,c_charge_desc_Batt/on a Specified Employee,c_charge_desc_Battery,c_charge_desc_Battery Emergency Care Provide,c_charge_desc_Battery On A Person Over 65,c_charge_desc_Battery On Fire Fighter,c_charge_desc_Battery On Parking Enfor Speci,c_charge_desc_Battery Spouse Or Girlfriend,c_charge_desc_Battery on Law Enforc Officer,c_charge_desc_Battery on a Person Over 65,c_charge_desc_Bribery Athletic Contests,c_charge_desc_Burgl Dwel/Struct/Convey Armed,c_charge_desc_Burglary Assault/Battery Armed,c_charge_desc_Burglary Conveyance Armed,c_charge_desc_Burglary Conveyance Assault/Bat,c_charge_desc_Burglary Conveyance Occupied,c_charge_desc_Burglary Conveyance Unoccup,c_charge_desc_Burglary Damage Property>$1000,c_charge_desc_Burglary Dwelling Armed,c_charge_desc_Burglary Dwelling Assault/Batt,c_charge_desc_Burglary Dwelling Occupied,c_charge_desc_Burglary Structure Armed,c_charge_desc_Burglary Structure Assault/Batt,c_charge_desc_Burglary Structure Occupied,c_charge_desc_Burglary Structure Unoccup,c_charge_desc_Burglary Unoccupied Dwelling,c_charge_desc_Burglary With Assault/battery,c_charge_desc_Carjacking w/o Deadly Weapon,c_charge_desc_Carjacking with a Firearm,c_charge_desc_Carry Open/Uncov Bev In Pub,c_charge_desc_Carrying A Concealed Weapon,c_charge_desc_Carrying Concealed Firearm,c_charge_desc_Cash Item w/Intent to Defraud,c_charge_desc_Cause Anoth Phone Ring Repeat,c_charge_desc_Child Abuse,c_charge_desc

In [12]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO IL DECISION TREE
dataset = X_test.copy(deep=True) # we create a copy of the test set
dataset['is_recid'] = y_test  # and join the target feature with the others
predictions = dataset.copy(deep=True) # we do the same task
predictions['is_recid'] = dt_predictions # but this time the target feature is made by the predictions of our model

In [21]:
fairness_metrics = compute_classification_metric(dataset,predictions,"is_recid",0,1,"sex_Female","sex_Male")
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.123
Equal Opportunity Difference (EOD): -0.069
Average Odds Difference: -0.141


In [26]:
#Calcolo le metriche di fairness considerando l'attributo razza come attributo sensibile
#Più precisamente, vengono calcolate considerando come:
#Attributo privilegiato la colonna race_Native_American
#Attributo non privilegiat la colonna race_african_American
#Valore di label favorevole 0 e label non favorevole 1

fairness_metrics = compute_classification_metric(dataset,predictions,"is_recid",0,1,"race_Native American","race_African-American")
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): 0.329
Equal Opportunity Difference (EOD): 0.244
Average Odds Difference: 0.352


In [18]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO L' SVM
dataset = X_test.copy(deep=True)
dataset['is_recid'] = y_test
predictions = dataset.copy(deep=True) 
predictions['is_recid'] = svm_pred 

In [27]:
fairness_metrics = compute_classification_metric(dataset,predictions,"is_recid",0,1,"sex_Male","sex_Female")
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.168
Equal Opportunity Difference (EOD): -0.118
Average Odds Difference: -0.172


In [28]:
fairness_metrics = compute_classification_metric(dataset,predictions,"is_recid",0,1,"race_Native American","race_African-American")
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): 0.329
Equal Opportunity Difference (EOD): 0.244
Average Odds Difference: 0.352


In [21]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO Il Random Forest
dataset = X_test.copy(deep=True)
dataset['is_recid'] = y_test
predictions = dataset.copy(deep=True) 
predictions['is_recid'] = rf_predictions 

In [29]:
fairness_metrics = compute_classification_metric(dataset,predictions,"is_recid",0,1,"sex_Male","sex_Female")
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.168
Equal Opportunity Difference (EOD): -0.118
Average Odds Difference: -0.172


In [30]:
fairness_metrics = compute_classification_metric(dataset,predictions,"is_recid",0,1,"race_Native American","race_African-American")
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): 0.329
Equal Opportunity Difference (EOD): 0.244
Average Odds Difference: 0.352
